In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


/tmp/ipykernel_13/1229163428.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Introduction

Assalamu alaikum everyone, this time our task is to create a model that does forecasting on sales in store in Ecuadorian.

Let's start with importing all libraries those we need.

In [10]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error, mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Dowloading data and creating data frames.

As you can see whole bunch of data. We should creat dataframe from each csv file and then convert date column to datatime from object.

In [11]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
train['date'] = pd.to_datetime(train['date'])

test  = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
test['date'] = pd.to_datetime(test['date'])

oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
oil['date'] = pd.to_datetime(oil['date'])

holidays_events = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
holidays_events['date'] = pd.to_datetime(holidays_events['date'])
holidays_events.rename(columns={'type': 'holiday_type'}, inplace=True)

stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
stores.rename(columns={'type': 'store_type'}, inplace=True)

transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
transactions['date'] = pd.to_datetime(transactions['date'])

# Creating new features.

In this cell here we are creating new featuers from date column, which are year, month, day, day of week, weekend. These features will be very usefull for our model.

In [12]:
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['day_of_week'] = train['date'].dt.dayofweek
train['weekend'] = train['day_of_week'].isin([5, 6]).astype(int)

test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['day_of_week'] = test['date'].dt.dayofweek
test['weekend'] = test['day_of_week'].isin([5, 6]).astype(int)

In Ecuador public sector pays the sales 2 times in a month, 15th day and the last day. So in following cell we will create column called 'payment_day', which describes wheater it is payment day.

In [13]:
train['payment_day'] = train['date'].apply(lambda x: 1 if x.day == 15 or x.is_month_end else 0)

test['payment_day'] = test['date'].apply(lambda x: 1 if x.day == 15 or x.is_month_end else 0)

A magnitude 7.8 earthquake struck Ecuador on April 16, 2016, so we create a feature informs about earthquake.

In [14]:
earth_quake = pd.to_datetime('2016-04-16 00:00:00')

train['after_earthquake'] = train['date'].apply(lambda x: 1 if x >= earth_quake else 1)

test['after_earthquake'] = test['date'].apply(lambda x: 1 if x >= earth_quake else 0)

# Merging main data to other datas.

In the following cells we will merge our main data with other data frames.

In [15]:
train = pd.merge(train, oil, on='date', how='left')
train['dcoilwtico'] = train['dcoilwtico'].fillna(method='backfill')

test = pd.merge(test, oil, on='date', how='left')
test['dcoilwtico'] = test['dcoilwtico'].fillna(method='backfill')

/tmp/ipykernel_13/3920736263.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['dcoilwtico'] = train['dcoilwtico'].fillna(method='backfill')
/tmp/ipykernel_13/3920736263.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test['dcoilwtico'] = test['dcoilwtico'].fillna(method='backfill')


In [16]:
train = pd.merge(train, holidays_events, how='left', on='date')

test = pd.merge(test, holidays_events, how='left', on='date')

In [17]:
train = pd.merge(train, stores, how='left', on='store_nbr')

test = pd.merge(test, stores, how='left', on='store_nbr')

In [18]:
train = pd.merge(train, transactions, how='left', on=['date', 'store_nbr'])

test = pd.merge(test, transactions, how='left', on=['date', 'store_nbr'])

In [19]:
train.drop(['date'], axis=1, inplace=True)

test.drop(['date'], axis=1, inplace=True)

# Label Encoding and Standart Scaling.

In [20]:
la = LabelEncoder()
cat_cols = ['family', 'holiday_type', 'locale', 'locale_name', 'transferred', 'city', 'state', 'store_type', 'description']

# Apply LabelEncoder to each categorical column separately
for col in cat_cols:
    train[col] = la.fit_transform(train[col])
    test[col] = la.transform(test[col])

# List of numerical columns
numerical_cols = ['year', 'cluster', 'day_of_week', 'month', 'weekend', 'payment_day', 'onpromotion', 'store_nbr', 'after_earthquake', 'transactions', 'dcoilwtico', 'day']

# Initialize StandardScaler
scaler = StandardScaler()

# Apply StandardScaler to the specified columns in train DataFrame
train[numerical_cols] = scaler.fit_transform(train[numerical_cols])

# Apply the same transformation to the specified columns in the test DataFrame
test[numerical_cols] = scaler.transform(test[numerical_cols])

# Spitting data.

In [21]:
X = train.drop(['sales'], axis=1)
y = train['sales']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# Training a model.

I chose XGB Regressor to our task.

In [ ]:
# Specify additional parameters
params = {
    'n_estimators': 1000,  # 1000 
    'max_depth': 20,       # 20
    'learning_rate': 0.1,  # You can adjust the learning rate
    'subsample': 0.8,      # Fraction of samples used for fitting the individual base learners
    'colsample_bytree': 0.8,  # Fraction of features used for fitting the individual base learners
    'gamma': 0,            # Minimum loss reduction required to make a further partition on a leaf node of the tree
    'objective': 'reg:squarederror',  # Specify the learning task and the corresponding objective function
    'reg_alpha': 0,        # L1 regularization term on weights
    'reg_lambda': 1,       # L2 regularization term on weights
    'random_state': 42,
    'n_jobs': -1
}

# Create XGBRegressor with specified parameters
xg_reg = xgb.XGBRegressor(**params)
xg_reg.fit(X_train, y_train)
Y_pred = xg_reg.predict(X_test)

squared_log_errors = (np.log1p(Y_pred) - np.log1p(y_test)) ** 2
mean_squared_log_error = np.mean(squared_log_errors)
rmsle = np.sqrt(mean_squared_log_error)
print("RMSLE:", rmsle)

# Preparing csv file to submission.

In [ ]:
test_pred = xg_reg.predict(test.drop('id', axis=1))

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'sales': test_pred})

In [ ]:
submission.to_csv('submission.csv', index=False)